<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/ANMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplos de ANMI

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols

import numpy as np
from numpy.linalg import cond as numero_condicion

from scipy.linalg import orth

## Funciones genéricas



In [ ]:
def norma(x):
  return sqrt((x.T * x)[0])

def print_verbose(msg, verbose):
  if type(msg) == str:
    msg = [msg]
  if verbose:
    for i in msg:
      print(i)

## Tema 2

### Número de condicion

El número de condición se define como $\vert\vert A \vert\vert \cdot \vert\vert A^{-1}\vert\vert$. Un número de condición cercano a 1 implica una matriz más estable frente a métodos con elementos diferenciales. Se cumple que las matrices ortogonales tienen número de condición 1.

In [ ]:
M = mat(((1, 2, 3), (2, 3, 1), (3, 2, 4)))
M

Matrix([
[1, 2, 3],
[2, 3, 1],
[3, 2, 4]])

In [ ]:
numero_condicion(np.array(M).astype(int))

6.960250455451664

In [ ]:
M_ort = (orth(np.array(M).astype(int)))
M_ort

array([[-0.50127996, -0.23313726,  0.83328592],
       [-0.45615124,  0.88953592, -0.02553206],
       [-0.73528528, -0.39290311, -0.55225239]])

In [ ]:
numero_condicion(M_ort)

1.0000000000000004

### Factorización LU y LDU*

Recordemos que para una matriz, la factorización LU es el proceso de aplicación de la simplificación de Gauss, de modo que la matriz $L$ es una matriz triangular inferior con los coeficientes de transformación, y la matriz $U$ es la matriz superior con los elementos tras las transformaciones lineales.

Además, se puede hacer que $D$ sea una matriz diagonal con los valores de la diagonal de $U$, de modo que $LU$ = $LDD^{-1}U$, y si hacemos $U^* = D^{-1}U$ entonces tenemos $LDU^*$, donde $U^*$ sigue siendo una matriz diagonal superior, pero con la diagonal igual a 1.

A la hora de aplicar la factorización LU y LDU* se puede hacer una permutación de filas, de modo que en cada iteración se coge la fila con mayor valor (de entre las que no se han *procesado*) y se permuta, garantizando una solución siempre. También, es importante tener en mente que la factorización falla si algún elemento de la diagonal (desde el principio o durante la factorización) es 0, de modo que para solucionar ese caso se aplica la permutación.

Todas las permutaciones quedan recogidas en una matriz $P$, de modo que $$LU = LDU^* = PA$$

In [ ]:
# Funciones asociadas a la factorización LU y LUD

def permutacion_matriz(U, fila_i, idx_max, verbose=False, P=None, r=None):
      print_verbose([f'Permutamos fila {fila_i} con {idx_max}',
                    f'U antes:\n {np.array(U)}',
                    f'P antes:\n {np.array(P)}'], verbose)

      if fila_i != idx_max:
        fila_origen, fila_destino = U[fila_i, :].copy(), U[idx_max, :].copy()
        U[idx_max, :], U[fila_i, :] = fila_origen, fila_destino
        if P is not None:
          fila_origen, fila_destino = P[fila_i, :].copy(), P[idx_max, :].copy()
          P[idx_max, :], P[fila_i, :] = fila_origen, fila_destino
        if r is not None:
          fila_origen, fila_destino = r[fila_i, :].copy(), r[idx_max, :].copy()
          r[idx_max, :], r[fila_i, :] = fila_origen, fila_destino

      print_verbose([f'U despues:\n {np.array(U)}',
                    f'P despues:\n {np.array(P)}'], verbose)

      return U, P, r


def permutacion_L(L, perm, verbose=False):
  """
  Esta función la creo porque a la hora de hacer la permutación, hay que 
  permutar los elementos de L pero no directamente. Solo hay que seleccionar
  los elementos de la diagonal inferior que correspondan con el menor número de
  columnas. Por ejemplo, si permutamos las filas 3 y 5, se tienen que mover solo
  los elementos de las 2 primeras columnas (2 -> 3) para luego continuar 
  con las transformaciones del resto de columnas.
  """
  print_verbose(f'L antes:\n {np.array(L)}', verbose)

  fila_origen, fila_destino = L[perm[0], :min(perm)].copy(), L[perm[1], :min(perm)].copy()
  L[perm[1], :min(perm)], L[perm[0], :min(perm)] = fila_origen, fila_destino

  print_verbose(f'L despues:\n {np.array(L)}', verbose)

  return L 


def descomposicion_LU(m, rhs=None, verbose=True, permutar_max=False):
  '''
  Esta función realiza el algoritmo de triangulación de Gauss. Para ello vamos a
  ir aplicando paso a paso el algoritmo tal cual se hace manualmente, 
  y aplicamos los cambios de columnas necesarios si hay que aplicar permutaciones.
  Por defecto, si encontramos un 0 en la diagonal aplicamos permutar_max para esa
  fila, y devolvemos la matriz de permutaciones.
  '''

  if rhs is None:
    rhs = zeros(m.shape[0], 1)

  print_verbose(["La matriz M|X es  (X = 0) si no se ha introducido", 
                np.concatenate((np.array(m), np.array(rhs)), axis=1)], verbose)
  
  P, L, U, r = eye(m.shape[0]), eye(m.shape[0]), m.copy(), rhs.copy()
  lista_perms = []
  fila_i, err, err_max = 0, 0, 3
  

  while fila_i < m.shape[0] and err < err_max:
    print_verbose([f'\n=====================================\nFila {fila_i}', 
                   f'A {fila_i}, {err}'], verbose)

    if U[max(0, fila_i-1), max(0, fila_i-1)] == 0 or permutar_max:
      """
      Esta parte nos asegura que si un elemento diagonal es cero, permutamos
      la fila con su inmediata inferior y rehacemos los cálculos, 
      y así tener una configuración viable.
      En cualquier permutación tenemos que cambiar la L acorde con el cambio.
      Al hacer la permutación, con los nuevos valores, repasamos la matriz para 
      asegurarnos de que todos los puntos están cumplidos.

      Para la opción de permutar, buscamos un elemento de las filas no alteradas
      que sea el mayor. Si hay más de una fila, y una de ellas es la 
      actual, no aplica la permutación.
      """
      sub_mat = U[fila_i:, fila_i:]
      max_el = np.max(np.array(sub_mat))
      idx_max = np.min([i[0] for i in np.argwhere(np.array(sub_mat) == max_el)]) + fila_i # Cogemos el primer elemento si hay varios
      
      if U[max(0, fila_i-1), max(0, fila_i-1)] == 0:
        U, P, r = permutacion_matriz(U, max(0, fila_i-1), idx_max, verbose, P, r)
        L = permutacion_L(L, [max(0, fila_i-1), idx_max], verbose)
        fila_i = min(max(0, fila_i-1), idx_max)
        
        err += 1
        print_verbose(f'Err {err}', verbose)
        continue

      elif permutar_max:
        if idx_max <= fila_i: # No es necesario hacer la permutación
          print_verbose('El índice de permutación es igual a la fila a permutar.', verbose)
        else:
          U, P, r = permutacion_matriz(U, fila_i, idx_max, verbose, P, r)
          L = permutacion_L(L, [fila_i, idx_max], verbose)
          continue

    # Ahora aplicamos el algoritmo de calculo de filas:
    for columna_j in range(fila_i):
      a_ij = U[fila_i, columna_j] /  U[columna_j, columna_j]
      
      if a_ij != 0:
        L[fila_i, columna_j] = a_ij
        U[fila_i, :] = U[fila_i, :] - a_ij * U[columna_j, :]
        r[fila_i, :] = r[fila_i, :] - a_ij * r[columna_j, :]

      err = 0

      print_verbose([
        f'||||||||||||||||||||||||\n Columna {columna_j}',
        f'a_{fila_i},{columna_j} = {a_ij}',
        f'P = \n{np.array(P)}',
        f'L = \n{np.array(L)}',
        f'U = \n{np.array(U)}',
        f'r = \n{np.array(r)}'], verbose)

    fila_i += 1
  
  if err == err_max:
    print('Algo ha ido mal... mira el log.')

  print_verbose([
        f'\/\/\/\/\/\/\/\/\/ FORMA FINAL',
        f'a_{fila_i},{columna_j} = {a_ij}',
        f'P = \n{np.array(P)}\n',
        f'L = \n{np.array(L)}\n',
        f'U = \n{np.array(U)}\n',
        f'r = \n{np.array(r)}\n'], verbose)

  if simplify(L * U) != simplify(P * m):
    print('AVISO!!! LU != PA')

  return {'P': simplify(P), 'L': simplify(L), 'U': simplify(U), 'r': simplify(r)}


def descomposicion_LDU(m, permutar_max=True, verbose=False):
  dict_LU = descomposicion_LU(m, permutar_max=permutar_max, verbose=verbose)
  L, U = dict_LU['L'], dict_LU['U']

  D = zeros(m.shape[0], m.shape[0])

  for i in range(U.shape[0]):
    D[i, i] = U[i, i]
  
  U = D.inv() * U

  print_verbose([
        f'\/\/\/\/\/\/\/\/\/ RESULTADOS LDU*',
        f'L = \n{np.array(L)}\n',
        f'D = \n{np.array(D)}\n',
        f'U* = \n{np.array(U)}\n'], verbose)

  if simplify(L * D * U) != simplify(dict_LU['P'] * m):
    print('AVISO!!! LDU != PA')

  return {'P': simplify(dict_LU['P']), 'L': simplify(L), 'U': simplify(U), 'D': simplify(D)}
  
  


In [ ]:
M = mat(((1, 4, 4), (3, 2, 1), (2, 4, 1)))

In [ ]:
descomposicion_LU(M, permutar_max=False)

La matriz M|X es  (X = 0) si no se ha introducido
[[1 4 4 0]
 [3 2 1 0]
 [2 4 1 0]]

Fila 0
A 0, 0

Fila 1
A 1, 0
||||||||||||||||||||||||
 Columna 0
a_1,0 = 3
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [0 0 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [2 4 1]]
r = 
[[0]
 [0]
 [0]]

Fila 2
A 2, 0
||||||||||||||||||||||||
 Columna 0
a_2,0 = 2
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [2 0 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [0 -4 -7]]
r = 
[[0]
 [0]
 [0]]
||||||||||||||||||||||||
 Columna 1
a_2,1 = 2/5
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [2 2/5 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [0 0 -13/5]]
r = 
[[0]
 [0]
 [0]]
\/\/\/\/\/\/\/\/\/ FORMA FINAL
a_3,1 = 2/5
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]

L = 
[[1 0 0]
 [3 1 0]
 [2 2/5 1]]

U = 
[[1 4 4]
 [0 -10 -11]
 [0 0 -13/5]]

r = 
[[0]
 [0]
 [0]]



{'L': Matrix([
 [1,   0, 0],
 [3,   1, 0],
 [2, 2/5, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]), 'U': Matrix([
 [1,   4,     4],
 [0, -10,   -11],
 [0,   0, -13/5]]), 'r': Matrix([
 [0],
 [0],
 [0]])}

In [ ]:
descomposicion_LDU(M, permutar_max=False)

{'D': Matrix([
 [1,   0,     0],
 [0, -10,     0],
 [0,   0, -13/5]]), 'L': Matrix([
 [1,   0, 0],
 [3,   1, 0],
 [2, 2/5, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]), 'U': Matrix([
 [1, 4,     4],
 [0, 1, 11/10],
 [0, 0,     1]])}

In [ ]:
descomposicion_LU(M, rhs=ones(M.shape[0], 1), permutar_max=True)

La matriz M|X es  (X = 0) si no se ha introducido
[[1 4 4 1]
 [3 2 1 1]
 [2 4 1 1]]

Fila 0
A 0, 0
El índice de permutación es igual a la fila a permutar.

Fila 1
A 1, 0
Permutamos fila 1 con 2
U antes:
 [[1 4 4]
 [3 2 1]
 [2 4 1]]
P antes:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
U despues:
 [[1 4 4]
 [2 4 1]
 [3 2 1]]
P despues:
 [[1 0 0]
 [0 0 1]
 [0 1 0]]
L antes:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
L despues:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]

Fila 1
A 1, 0
El índice de permutación es igual a la fila a permutar.
||||||||||||||||||||||||
 Columna 0
a_1,0 = 2
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[[1 0 0]
 [2 1 0]
 [0 0 1]]
U = 
[[1 4 4]
 [0 -4 -7]
 [3 2 1]]
r = 
[[1]
 [-1]
 [1]]

Fila 2
A 2, 0
El índice de permutación es igual a la fila a permutar.
||||||||||||||||||||||||
 Columna 0
a_2,0 = 3
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[[1 0 0]
 [2 1 0]
 [3 0 1]]
U = 
[[1 4 4]
 [0 -4 -7]
 [0 -10 -11]]
r = 
[[1]
 [-1]
 [-2]]
||||||||||||||||||||||||
 Columna 1
a_2,1 = 5/2
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[

{'L': Matrix([
 [1,   0, 0],
 [2,   1, 0],
 [3, 5/2, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0]]), 'U': Matrix([
 [1,  4,    4],
 [0, -4,   -7],
 [0,  0, 13/2]]), 'r': Matrix([
 [  1],
 [ -1],
 [1/2]])}

In [ ]:
descomposicion_LDU(M, permutar_max=True)

{'D': Matrix([
 [1,  0,    0],
 [0, -4,    0],
 [0,  0, 13/2]]), 'L': Matrix([
 [1,   0, 0],
 [2,   1, 0],
 [3, 5/2, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0]]), 'U': Matrix([
 [1, 4,   4],
 [0, 1, 7/4],
 [0, 0,   1]])}

### Factorización de Cholesky

La factorización de Cholesky es una factorización que genera una matriz triangular inferior $L$ tal que $A = LL^T$. Para que una matriz sea factorizable, tiene que cumplir que sus menores principales sean positivos, y que sea simétrica.

In [ ]:
def cholesky(m, verbose=False):
  """
  Primero comprobamos que los menores sean positivos. Eso es equivalente que
  sus autovalores sean positivos. Por simplificar, si el determinante es negativo
  ya descartamos que sea factorizable, y saltamos el warning.
  """

  if m != m.T:
    print('AVISO! La matriz no es simétrica, y por tanto no factorizable por Cholesky.')

  print_verbose(f"|M| es {m.det()}. Si es < 0, no es factorizable.", verbose)

  m_chol = zeros(m.shape[0], m.shape[1])

  for col_j in range(m.shape[0]):
    for row_i in range(col_j, m.shape[0]):
      if col_j == 0:
        if row_i == 0:
          m_chol[row_i, col_j] = sqrt(m[row_i, col_j])
        else:
          m_chol[row_i, col_j] = m[row_i, col_j] / m_chol[0, 0]
      
      else:
        if col_j == row_i:
          m_chol[row_i, col_j] = sqrt(m[col_j, col_j] - sum([m_chol[col_j, k] 
                                                      ** 2 for k in range(col_j)]))
        else:
          m_chol[row_i, col_j] = (m[row_i, col_j] - sum([m_chol[col_j, k] * m_chol[row_i, k] for k in range(col_j)]))/(m_chol[col_j, col_j])

  if simplify(m_chol * m_chol.T) != m:
    print(f"AVISO!!! La matriz no es Cholesky-zable. \n L = \n {np.array(m_chol)} \n\n L*L.T = \n {np.array(m_chol * m_chol.T)}")

  return simplify(m_chol)

In [ ]:
M = mat(((2, -1, 0), (-1, 2, -2), (0, 2, 1)))
M

Matrix([
[ 2, -1,  0],
[-1,  2, -2],
[ 0,  2,  1]])

In [ ]:
cholesky(M)

AVISO! La matriz no es simétrica, y por tanto no factorizable por Cholesky.
AVISO!!! La matriz no es Cholesky-zable. 
 L = 
 [[sqrt(2) 0 0]
 [-sqrt(2)/2 sqrt(6)/2 0]
 [0 2*sqrt(6)/3 sqrt(15)*I/3]] 

 L*L.T = 
 [[2 -1 0]
 [-1 2 2]
 [0 2 1]]


Matrix([
[   sqrt(2),           0,            0],
[-sqrt(2)/2,   sqrt(6)/2,            0],
[         0, 2*sqrt(6)/3, sqrt(15)*I/3]])

In [ ]:
cholesky(M + M.T)

Matrix([
[ 2,       0,       0],
[-1, sqrt(3),       0],
[ 0,       0, sqrt(2)]])

In [ ]:
M + M.T

Matrix([
[ 4, -2, 0],
[-2,  4, 0],
[ 0,  0, 2]])

In [ ]:
cholesky(M + M.T) * cholesky(M + M.T).T

Matrix([
[ 4, -2, 0],
[-2,  4, 0],
[ 0,  0, 2]])

In [ ]:
# Podemos hacer también Cholesky a una matriz con símbolos!
x = symbols('x')

Mx = mat(((1, x, 1), (x, 2, 1), (1, 1, 3)))
Mx

Matrix([
[1, x, 1],
[x, 2, 1],
[1, 1, 3]])

In [ ]:
cholesky(Mx)

Matrix([
[1,                      0,                                    0],
[x,         sqrt(2 - x**2),                                    0],
[1, (1 - x)/sqrt(2 - x**2), sqrt((-3*x**2 + 2*x + 3)/(2 - x**2))]])

### Ortogonalización de Gram-Schmidt




In [ ]:
def suma_columnas(lista):
  if len(lista) > 0:
    m = zeros(lista[0].shape[0], 1)
    for i in lista:
      m += i
    return m
  else:
    return 0

def gram_schmidt(m, verbose=False):
  """
  La ortogonalización produce una matriz ortogonalizada por columnas.
  p es la matriz ortogonal y p_norm es la ortonormal
  c es la matriz triangular tal que cij = aj·pi/||pi||^2 (los coeficientes de ortogonalización). Estos coeficientes se usan para la factorización QR.
  """

  p = zeros(m.shape[0], m.shape[1])

  p[:, 0] = m[:, 0]

  for col in range(1, m.shape[1]):
    p[:, col] = m[:, col] - suma_columnas([(m[:, col].T * p[:, i])[0]/(p[:, i].T * p[:, i])[0] * p[:, i] for i in range(0, col)])

  print_verbose(f"La matriz ortogonal es \n {p}", verbose)


  p_norm = zeros(m.shape[0], m.shape[1])
  for col in range(p.shape[1]):
    p_norm[:, col] = p[:, col] / (p[:, col].T * p[:, col])[0]
  
  print_verbose(f"La matriz ortonormal es \n {p_norm}", verbose)


  c = zeros(m.shape[0], m.shape[1])
  for col in range(1, m.shape[1]):
    for row in range(0, col):
      c[row, col] = (m[:, col].T * p[:, row])[0]/(p[:, row].T * p[:, row])[0]

  return {'P': simplify(p), 'Pn': simplify(p_norm), 'c': simplify(c)} 


In [ ]:
M

Matrix([
[ 2, -1,  0],
[-1,  2, -2],
[ 0,  2,  1]])

In [ ]:
GS = gram_schmidt(M)
GS['P']

Matrix([
[ 2, 3/5, -22/29],
[-1, 6/5, -44/29],
[ 0,   2,  33/29]])

In [ ]:
GS['Pn']

Matrix([
[ 2/5,  3/29, -2/11],
[-1/5,  6/29, -4/11],
[   0, 10/29,  3/11]])

In [ ]:
GS['c']

Matrix([
[0, -4/5,   2/5],
[0,    0, -2/29],
[0,    0,     0]])

In [ ]:
GS['P'][:, 1].T * GS['P'][:, 2]

Matrix([[0]])

In [ ]:
Mx = mat(((1, 2, 3), (1, x, 1), (0, 0, 3)))

In [ ]:
GSx = gram_schmidt(Mx)
GSx['P']

Matrix([
[1, 1 - x/2, 0],
[1, x/2 - 1, 0],
[0,       0, 3]])

### Transformación de Householder

La transformación de Householder es una transformación para pasar de un vertor $x$ a un vector $y$. Para ello se toma el vector $e$, que sería el eje de transformación, y la matriz aplicación es $H = I - 2ee^t$.

En este caso, $$e = \pm \frac{x - y}{\vert\vert x - y \vert\vert}$$

Para la transformación de Householder $x$ e $y$ **tienen que tener la misma norma**. El vector resultante $e$ tiene norma 1.


In [ ]:
def householder(x, y, signo='+', normalizar = False):
  """
  Aplica la transformación de Householder. x e y pueden ser vectores fila o columna, 
  en cuyo caso aplicamos la transformación de una forma u otra.
  """
  if signo == '+':
    mult = 1
  else:
    mult = -1

  if normalizar:
    x = simplify(x / norma(x))
    y = simplify(y / norma (y))

  if simplify(norma(x)) != simplify(norma(y)):
    print(f'AVISO!!! x tiene norma {norma(x)} e y tiene norma {norma(y)}. Para Householder las normas tienen que ser iguales.')

  if simplify(norma(x - y)) == 0:  # x==y
    e = zeros(x.shape[0], x.shape[1])
  else:
    e = mult * (x - y)/(norma(x - y))

  if (e.shape[0] == 1) and (e.shape[1] >= 1): # El vector es fila:
    H = eye(x.shape[1]) - 2 * e.T * e
  else:
    H = eye(x.shape[0]) - 2 * e * e.T
  

  if (e.shape[0] == 1) and (e.shape[1] >= 1) and (simplify(x * H) != y):
    print(f'AVISO FILAS!!! xH != y. Para Householder las normas tienen que ser iguales.')
  elif (e.shape[1] == 1) and (e.shape[0] >= 1) and (simplify(H * x) != y):
    print(f'AVISO!!! Hx != y. Para Householder las normas tienen que ser iguales.')

  return simplify(H), simplify(e)

In [ ]:
v0 = mat(((1, 0)))
vf = mat((1, 1))
vf = vf/norma(vf)

In [ ]:
H, e = householder(v0, vf, normalizar=False)
H

Matrix([
[sqrt(2)/2,  sqrt(2)/2],
[sqrt(2)/2, -sqrt(2)/2]])

In [ ]:
e

Matrix([
[           sqrt(2 - sqrt(2))/2],
[-sqrt(2)/(2*sqrt(2 - sqrt(2)))]])

In [ ]:
# Ejercicio 10
t = symbols('t')

v0 = mat(((1, 0)))
vf = mat((cos(t), sin(t)))

H, e = householder(v0, vf, normalizar=False)

In [ ]:
H

Matrix([
[cos(t),  sin(t)],
[sin(t), -cos(t)]])

In [ ]:
e

Matrix([
[      sqrt(2 - 2*cos(t))/2],
[-sin(t)/sqrt(2 - 2*cos(t))]])

### Factorización QR

La factorización QR consiste en transformar $A = QR$ donde $Q$ es ortogonal y $R$ es triangular superior. 

Si $P$ es la matriz ortogonalizada, $C$ es la matriz con los factores de ortonormalización (para Gram-Schmidt, por ejemplo, es $m_{ij} = \frac{a^j\cdot p^i}{\vert\vert p^{i}\vert\vert^2}$ y $D$ es la matriz de las normas de los vectores ortogonales ($\vert\vert p^i\vert\vert$), entonces se tiene que:
$$Q = PD^{-1}$$
$$R = D(I + C)$$

Para el método de Hausholder, vamos a ir por columnas. Para cada columna, seleccionamos la submatriz $N(i) = A_{ii}$ de (n-i) filas y columnas. Para esa matriz, vamos a hacer que la primera columna, $p_i$ pase a ser $\vert\vert p_i\vert\vert(

In [ ]:
TODO: ESCRIBIR DOCU Y MEJORAR TEXTO/USER FIRENDLYNESS

In [163]:
def factorizacion_QR(m, verbose=True, metodo='gram_schmidt'):
  if metodo == 'gram_schmidt':
    print_verbose('Aplicamos QR con Gram Schmidt', verbose)

    dict_gs = gram_schmidt(m)
    P, C = dict_gs['P'], dict_gs['c']
  
    D = zeros(m.shape[0], m.shape[1])
    for col in range(m.shape[0]):
      D[col, col] = sqrt((P[:, col].T * P[:, col])[0])
    
    Q = simplify(P * (D ** (-1)))
    R = simplify(D * (eye(m.shape[0]) + C))
    
    print_verbose(f'Q es \n{np.array(Q)}\nR es \n{np.array(R)}\nD es \n{np.array(D)}', verbose)
  elif metodo == 'householder':
    print_verbose('Aplicamos QR con Householder', verbose)
    lista_p = [eye(m.shape[0]) for _ in range(m.shape[0])]
    lista_e = []

    Q, R = eye(m.shape[0]), m.copy()
    for col in range(m.shape[1]):
      x = R[col:, col]
      y = mat([1] + [0] * (m.shape[0] - col - 1)) * norma(x)
      p_col = lista_p[col]
      H, e = householder(x, y)
      p_col[col:, col:] = H
      lista_e.append(e)
      
      R = simplify(p_col * R)
      Q = simplify(Q * p_col)

    print_verbose(f'Q es \n{np.array(Q)}\nR es \n{np.array(R)}', verbose)
  else:
    raise "El método no existe. Los disponibles son gram_schmidt y householder"
  

  if m != simplify(Q * R):
    print('AVISO!!! A != QR')

  if metodo=='gram_schmidt':
    return {'Q': Q, 'R': R, 'D': D} 
  elif metodo == 'householder':
    return {'Q': Q, 'R': R} 

In [152]:
M = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [154]:
factorizacion_QR(M, metodo='householder')

Aplicamos QR con Householder
Q es 
[[1 0 0]
 [0 0 -1]
 [0 1 0]]
R es 
[[2 -1 0]
 [0 2 -1]
 [0 0 2]]


{'Q': Matrix([
 [1, 0,  0],
 [0, 0, -1],
 [0, 1,  0]]), 'R': Matrix([
 [2, -1,  0],
 [0,  2, -1],
 [0,  0,  2]])}

In [ ]:
A1 = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [155]:
A1 = mat(((1, 2, 3), (4, 5, 6), (7, 8, 9)))
A1

Matrix([
[1, 2, 3],
[4, 5, 6],
[7, 8, 9]])

In [164]:
dqr = factorizacion_QR(A1, metodo='householder')

Aplicamos QR con Householder
AVISO!!! Hx != y. Para Householder las normas tienen que ser iguales.
Q es 
[[sqrt(66)/66
  3*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))
  (-72 - 7*sqrt(66) + 65*sqrt(6))/(6*(-65 + 7*sqrt(11) + 12*sqrt(6)))]
 [2*sqrt(66)/33
  (-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))
  (-65*sqrt(6) + 7*sqrt(66) + 72)/(3*(-65 + 7*sqrt(11) + 12*sqrt(6)))]
 [7*sqrt(66)/66
  (-12*sqrt(66) - 77 + 65*sqrt(11))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))
  (-72 - 7*sqrt(66) + 65*sqrt(6))/(6*(-65 + 7*sqrt(11) + 12*sqrt(6)))]]
R es 
[[sqrt(66) 13*sqrt(66)/11 15*sqrt(66)/11]
 [0
  3*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))
  6*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))]
 [0 0 0]]


In [165]:
dqr['Q']

Matrix([
[  sqrt(66)/66, 3*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6))), (-72 - 7*sqrt(66) + 65*sqrt(6))/(6*(-65 + 7*sqrt(11) + 12*sqrt(6)))],
[2*sqrt(66)/33,   (-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6))), (-65*sqrt(6) + 7*sqrt(66) + 72)/(3*(-65 + 7*sqrt(11) + 12*sqrt(6)))],
[7*sqrt(66)/66,   (-12*sqrt(66) - 77 + 65*sqrt(11))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6))), (-72 - 7*sqrt(66) + 65*sqrt(6))/(6*(-65 + 7*sqrt(11) + 12*sqrt(6)))]])

In [166]:
dqr['R']

Matrix([
[sqrt(66),                                                           13*sqrt(66)/11,                                                           15*sqrt(66)/11],
[       0, 3*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6))), 6*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))],
[       0,                                                                        0,                                                                        0]])

In [161]:
simplify(dqr['Q'].T * dqr['Q'])

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [162]:
simplify(dqr['Q'] * dqr['R'])

Matrix([
[1, 2, 3],
[4, 5, 6],
[7, 8, 9]])

### Métodos iterativos

TODO: GRAFICAR ITERACIONES, PRINT_VERBOSES ETC

In [261]:
def norma_1(A):
  print('La norma 1 es: ', max(np.sum(abs(A), axis=0))) # Mayor columna

def norma_inf(A):
  print('La norma inf es: ', max(np.sum(abs(A), axis=1))) # Mayor fila

def norma_2(A):
  eigs = sqrt([simplify(i) for i in list((A.T * A).eigenvals().keys())])
  print('Los eigs de A.T * A (sqrt) son:', eigs)
  print('La norma 2 es: ', np.max(eigs))

In [246]:
A = mat([[1, a, 3], [3,2,2], [3,6,7]])
np.sum(A, axis=1)

array([a + 4, 7, 16], dtype=object)

In [236]:
def criterio_radio_espectral(H, verbose=True):
  eigs = [simplify(i) for i in list(H.eigenvals().keys())]

  print_verbose('||Criterio de radio espectral||', verbose)
  try:
    print_verbose(f'El mayor autovalor es {np.max(np.array(eigs, dtype=float))}. Si ese valor es < 1 entonces los métodos iterativos convergen.', verbose)
  except:
    print_verbose(f'Los autovalores son {eigs}. Si el mayor autovalor es < 1, entonces el método converge.', verbose)

def criterio_diagonal_dominante(A, verbose=True):
  print_verbose('||Criterio de Diagonal Dominante||\n Si la matriz es dominante por filas, los métodos de Jacobi y Gauss-Seidel convergen.', verbose)
  A_abs = abs(A)
  try:
    np.array(A_abs, dtype=float)
    for r in range(A.shape[0]):
      diff = 2 * A_abs[r, r] - sum(A_abs[r, :])
      if diff <=  0:
        print_verbose(f'La fila {r} NO es dominante por filas: diff = {diff}.', verbose)
        return
    print_verbose('La matriz CUMPLE EL CRITERIO DIAGONAL DOMINANTE', verbose)
  except:
    print_verbose('La matriz tiene complejos o simbolos. Hay que verificar el criterio a mano.', verbose)
  
def criterio_simetrica_definida_positiva(A, verbose=True):
  print_verbose('||Criterio de Sim Def Pos||\n Si la matriz es simétrica y definida positiva, el método de Gauss-Seidel es convergente.', verbose)
  if A != A.T:
    print_verbose('La matriz NO es simétrica.', verbose)
    return 
  
  det_A = A.det()
  print_verbose(f'El determinante de A es {det_A}.', verbose)
  try:
    if float(det_A) > 0:
      print_verbose(f'La matriz es DEFINIDA POSITIVA (el determinante es positivo).', verbose)
      print_verbose('La matriz CUMPLE EL CRITERIO SIM DEF POS', verbose)
    else:
      print_verbose(f'La matriz NO es DEFINIDA POSITIVA (el determinante no es positivo).', verbose)
  except:
    print_verbose(f'No podemos determinar la positividad porque hay símbolos o complejos.', verbose)

def criterio_SOR(verbose):
  print_verbose('||Criterio SOR||\n Si la matriz es simétrica y definida positiva y w in (0, 2) el método SOR es convergente.\nSi w no (0, 2) el método SOR no converge.', verbose)

def criterio_m_matriz(A, verbose):
  print_verbose('||Criterio M matriz||\n Si la A es M-matriz entonces las descomposiciones de Jacobi y Gauss-Seidel son convergentes.\nA^-1 >= 0\naij < 0 para todo i =/= j', verbose)
  A_inv = A ** -1

  try:
    np.array(A, dtype=float)
    if np.min(A_inv) >= 0:
      print_verbose('A^-1 >= 0', verbose)
    else:
      print_verbose('A^-1 < 0. La matriz NO CUMPLE el criterio', verbose)
    
    A_null_diag = A.copy()
    for i in range(A.shape[0]):
      A_null_diag[i, i] = 0
    
    if np.max(A_null_diag) > 0:
      print_verbose('La matriz tiene elementos no diagonales positivos. NO CUMPLE el criterio.', verbose)
    else:
      print_verbose('Los elementos no diagonales son negativos.', verbose)
  except:
    print_verbose('La matriz tiene complejos o símbolos, no podemos verificar le criterio.', verbose)


In [227]:
def metodo_iterativo(A, b=None, x0=None, metodo='jacobi', w=1.5, n_iter=10, verbose=True, grafica_conver=True):
  if b is None:
    b = mat([[1] * A.shape[0]]).T
  if x0 is None:
    x0 = mat([[1] * A.shape[1]]).T

  D, L, U = zeros(A.shape[0], A.shape[1]), zeros(A.shape[0], A.shape[1]), zeros(A.shape[0], A.shape[1])
  for r in range(A.shape[0]):
    for c in range(A.shape[1]):
      if r == c:
        D[r, c] = A[r, c]
      elif r < c:
        U[r, c] = - A[r, c]
      else:
        L[r, c] = - A[r, c]

  if metodo=='jacobi':
    M = D
  elif metodo=='gs':
    M = D - L
  elif metodo=='sor':
    M = D/w - L
  
  N = simplify(M - A)

  # Aplicamos criterios!
  criterio_radio_espectral(M ** (-1) * N, verbose)
  criterio_diagonal_dominante(A, verbose)
  criterio_simetrica_definida_positiva(A, verbose)
  criterio_SOR(verbose)
  criterio_m_matriz(A, verbose)

  diff = []
  for iter in range(n_iter):
    x0 = (M ** (-1)) * (N * x0 + b)
    diff.append(np.sum(np.abs(A * x0 - b)))

  return {'x': x0, 'diff': diff}

In [233]:
A = mat([[2, 1, 1], [1, -2, 1], [1, 1, 2]])
b = mat([[1, 1, 1]]).T

In [237]:
diter = metodo_iterativo(A, b, n_iter=200, verbose=True)
np.array(diter['x'], dtype=float)

||Criterio de radio espectral||
Los autovalores son [1/2, -1/4 - sqrt(7)*I/4, -1/4 + sqrt(7)*I/4]. Si el mayor autovalor es < 1, entonces el método converge.
||Criterio de Diagonal Dominante||
 Si la matriz es dominante por filas, los métodos de Jacobi y Gauss-Seidel convergen.
La fila 0 NO es dominante por filas: diff = 0.
||Criterio de Sim Def Pos||
 Si la matriz es simétrica y definida positiva, el método de Gauss-Seidel es convergente.
El determinante de A es -8.
La matriz NO es DEFINIDA POSITIVA (el determinante no es positivo).
||Criterio SOR||
 Si la matriz es simétrica y definida positiva y w in (0, 2) el método SOR es convergente.
Si w no (0, 2) el método SOR no converge.
||Criterio M matriz||
 Si la A es M-matriz entonces las descomposiciones de Jacobi y Gauss-Seidel son convergentes.
A^-1 >= 0
aij < 0 para todo i =/= j
A^-1 < 0. La matriz NO CUMPLE el criterio
La matriz tiene elementos no diagonales positivos. NO CUMPLE el criterio.


array([[ 0.375],
       [-0.125],
       [ 0.375]])

In [231]:
# EJERCICIO 12
a = symbols('a')

A = mat([[4, 1, a], [1, 4, 1], [a, 1, 4]])
metodo_iterativo(A, metodo='gs', n_iter=10)

||Criterio de radio espectral||
Los autovalores son [a**2/32 - a/128 - sqrt((4*a - 1)*(4*a**3 - a**2 + 16*a - 64))/128 + 1/16, a**2/32 - a/128 + sqrt((4*a - 1)*(4*a**3 - a**2 + 16*a - 64))/128 + 1/16, 0]. Si el mayor autovalor es < 1, entonces el método converge.
||Criterio de Diagonal Dominante||
 Si la matriz es dominante por filas, los métodos de Jacobi y Gauss-Seidel convergen.
La matriz tiene complejos o simbolos. Hay que verificar el criterio a mano.
||Criterio de Sim Def Pos||
 Si la matriz es simétrica y definida positiva, el método de Gauss-Seidel es convergente.
El determinante de A es -4*a**2 + 2*a + 56.
No podemos determinar la positividad porque hay símbolos o complejos.
||Criterio SOR||
 Si la matriz es simétrica y definida positiva y w in (0, 2) el método SOR es convergente.
Si w no (0, 2) el método SOR no converge.
||Criterio M matriz||
 Si la A es M-matriz entonces las descomposiciones de Jacobi y Gauss-Seidel son convergentes.
A^-1 >= 0
aij < 0 para todo i =/= j


TypeError: ignored

In [270]:
# EJERCICIO 14
A = mat([[-11, 20, 8], [20, 16, -8], [8, -8, 5]]) / 27
norma_1(A)
norma_inf(A)
d = [simplify(i) for i in list((A.T * A).eigenvals().keys())]
# norma_2(A)

La norma 1 es:  44/27
La norma inf es:  44/27


In [285]:
A

Matrix([
[-11/27, 20/27,  8/27],
[ 20/27, 16/27, -8/27],
[  8/27, -8/27,  5/27]])

In [284]:
A.T * A

Matrix([
[   65/81,   4/81, -208/729],
[    4/81,  80/81,   -8/729],
[-208/729, -8/729,    17/81]])

In [282]:
simplify((A.T * A - a * eye(3)))

Matrix([
[65/81 - a,      4/81,  -208/729],
[     4/81, 80/81 - a,    -8/729],
[ -208/729,    -8/729, 17/81 - a]])

In [272]:
simplify(d[0])

2/3 - (233455689 + 24720*sqrt(13236603)*I)**(1/3)/2187 - 132347*3**(2/3)/(2187*(77818563 + 8240*sqrt(13236603)*I)**(1/3))